<h1>BÀI TẬP LỚN Môn : Xử Lý Ngôn Ngữ Tự Nhiên</h1>

-----------------------

*Nhóm 21 - Chủ đề : Tóm tắt văn bản (Text Summarization)*
-----------------------
* Nguyễn Việt Anh - 2121050054
* Phạm Thị Liên - 212151542
* Lê Danh Cần - 2121051529

<h2>Extractive Summarization</h2>
<h3>Sentence Scoring</h3>

In [45]:
import nltk
from underthesea import word_tokenize
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


with open('Data/output_2024-05-28_11-20-11.txt', 'r', encoding='utf-8') as file:
    text = file.read()


with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# Hàm xử lý văn bản
def text_handle(text):
    lower_text = text.lower()
    clean_text = re.sub(r'\s+', ' ', lower_text)
    return clean_text.strip()

# Hàm loại bỏ từ dừng
def remove_stopwords(sentence, stopwords):
    words = word_tokenize(sentence)
    filtered_sentence = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_sentence)

# Hàm loại bỏ dấu câu
def remove_punctuation(sentence):
    sentence = re.sub(r'\d+', '', sentence)
    return re.sub(r'[^\w\s]', '', sentence)

# Tách các câu trong văn bản
original_sentences = nltk.sent_tokenize(text_handle(text))
print("Các câu ban đầu:")
print(original_sentences)

# Xử lý từng câu
processed_sentences = []
for sentence in original_sentences:
    cleaned_sentence = remove_stopwords(sentence, stopwords)
    cleaned_sentence = remove_punctuation(cleaned_sentence)
    cleaned_sentence = ' '.join(cleaned_sentence.split())  # Loại bỏ khoảng trắng thừa
    processed_sentences.append(cleaned_sentence)

print("Các câu sau khi xử lý:")
print(processed_sentences)

Các câu ban đầu:
["đề xuất người lao động đóng bù khoản bhxh bị 'trốn' để hưởng chế độ - báo vnexpress trong dự thảo trình quốc hội ngày 27/5, ban soạn thảo và cơ quan thẩm tra đề xuất bổ sung cơ chế bảo vệ lao động trong trường hợp người sử dụng không còn khả năng đóng như phá sản, giải thể, bỏ trốn.", 'cơ quan bhxh sẽ tạm thời xác nhận thời gian đã đóng nếu lao động có yêu cầu để làm căn cứ hưởng các chế độ "đóng đến đâu tính đến đó".', 'khi chủ sử dụng đóng bù thì lao động sẽ được xác nhận, cộng thời gian này vào để tính lại.', 'trường hợp tính cả thời gian bị chậm, trốn đóng bhxh mới đủ điều kiện hưởng hưu trí và các chế độ khác thì lao động có thể chọn nộp khoản bị chậm, trốn này vào quỹ hưu trí tử tuất để được xác nhận hưởng.', 'đại biểu nguyễn hoàng bảo trân (phó chủ tịch liên đoàn lao động tỉnh bình dương) băn khoăn việc lao động phải đóng bù tiền để hưởng quyền lợi mà họ đương nhiên được hưởng.', 'khi doanh nghiệp chậm, trốn đóng bhxh, người lao động đã mất trắng khoản lương h

In [46]:
# Hàm tính điểm TF-IDF cho từng câu
def calculate_tfidf(sentences):
    tfidf_vector = TfidfVectorizer()
    tfidf_matrix = tfidf_vector.fit_transform(sentences)
    feature_names = tfidf_vector.get_feature_names_out()
    df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
    print('Điểm các từ trong câu: \n', df)
    tfidf_score = np.asarray(tfidf_matrix.sum(axis=1)).ravel()
    return tfidf_score

# Hàm tính điểm cho mỗi câu dựa trên chiều dài và điểm TF-IDF
def calculate_sentence_scores(sentences):
    tfidf = calculate_tfidf(sentences)
    print('Ma trận điểm TF-IDF: \n', tfidf)
    sentence_lengths = [len(sentence.split()) for sentence in sentences]
    print('Ma trận độ dài từng câu: \n', sentence_lengths)
    max_sentence_length = max(sentence_lengths)
    normalized_lengths = [length / max_sentence_length for length in sentence_lengths]
    print('Điểm chuẩn hóa câu trong văn bản: \n', normalized_lengths)
    sentence_scores = [tfidf_score * length for tfidf_score, length in zip(tfidf, normalized_lengths)]
    return sentence_scores

sentence_scores = calculate_sentence_scores(processed_sentences)

# Hiển thị điểm của từng câu
print("Điểm của từng câu:")
print(sentence_scores)

Điểm các từ trong câu: 
           an       ban      bhxh      binh      biểu      biện      buộc  \
0   0.000000  0.148034  0.085575  0.000000  0.000000  0.000000  0.000000   
1   0.000000  0.000000  0.167796  0.000000  0.000000  0.000000  0.000000   
2   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3   0.000000  0.000000  0.106497  0.000000  0.000000  0.000000  0.000000   
4   0.000000  0.000000  0.000000  0.000000  0.164368  0.000000  0.000000   
5   0.000000  0.284853  0.164667  0.000000  0.000000  0.000000  0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
8   0.000000  0.000000  0.168357  0.000000  0.000000  0.329582  0.000000   
9   0.324287  0.000000  0.000000  0.000000  0.239022  0.000000  0.000000   
10  0.000000  0.000000  0.209299  0.000000  0.000000  0.000000  0.000000   
11  0.000000  0.000000  0.226829  0.000000  0.000000  0.000000 

In [47]:
def summarize_text(original_sentences, sentence_scores, num_sentences):
    # Sắp xếp các câu theo điểm số giảm dần
    sorted_indices = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)
    # Chọn ra các câu có điểm cao nhất để tạo tóm tắt
    summary_indices = sorted_indices[:num_sentences]
    # Sắp xếp lại các chỉ số câu theo thứ tự xuất hiện trong văn bản
    summary_indices.sort()
    # Tạo tóm tắt bằng cách lấy các câu gốc được chọn
    summary = [original_sentences[i] for i in summary_indices]
    return summary

num_summary_sentences = 3
summary = summarize_text(original_sentences, sentence_scores, num_summary_sentences)

print("Văn bản tóm tắt:")
for sentence in summary:
    print(sentence)


Văn bản tóm tắt:
đề xuất người lao động đóng bù khoản bhxh bị 'trốn' để hưởng chế độ - báo vnexpress trong dự thảo trình quốc hội ngày 27/5, ban soạn thảo và cơ quan thẩm tra đề xuất bổ sung cơ chế bảo vệ lao động trong trường hợp người sử dụng không còn khả năng đóng như phá sản, giải thể, bỏ trốn.
bà trân đề nghị đánh giá hiệu quả của hoạt động thanh kiểm tra của cơ quan bhxh, tránh tái diễn việc như thu tiền bhxh sai đối tượng, điển hình như hơn 4.000 chủ hộ kinh doanh cá thể đến nay vẫn chưa được giải quyết dứt điểm quyền lợi.
trước đó, tổng liên đoàn lao động việt nam cũng kiến nghị thủ tướng sớm chỉ đạo bộ lao động thương binh và xã hội rà soát, có chính sách giải quyết quyền lợi bị "treo" cho hơn 213.000 lao động trong doanh nghiệp phá sản, giải thể, chủ bỏ trốn.
